# Bernoulli Console With Random Orientation

In [1]:
L, d, t = 1000.0, 100.0, 5.0
Ex, nu = 210000.0, 0.25
Fz = 1000.0
n = 2  # number of element
penalty = 1e20  # penalty value


In [2]:
seed = 0  # integer or None
seed = None


## Linear Solution

In [3]:
from linkeddeepdict import LinkedDeepDict
from linkeddeepdict.tools import getallfromkwargs
from neumann.linalg import Vector, linspace
from neumann.array import repeat
from polymesh.space import StandardFrame, PointCloud, frames_of_lines
from polymesh.space.utils import index_of_closest_point, index_of_furthest_point
from polymesh.topo.tr import L2_to_L3
from sigmaepsilon.solid.fem.cells import B2, B3
from sigmaepsilon.solid import Structure, LineMesh, PointData, BeamSection
import numpy as np
from latexdocs.utils import floatformatter

rs = np.random.RandomState()
if isinstance(seed, int):
    rs.seed(seed)

formatter = floatformatter(sig=6)
def f2s(x): return formatter.format(x)


# section
section = BeamSection('CHS', d=d, t=t, n=32)
section.calculate_section_properties()
section_props = section.section_properties
A, Ix, Iy, Iz = getallfromkwargs(['A', 'Ix', 'Iy', 'Iz'], **section_props)

# material
G = Ex / (2 * (1 + nu))
Hooke = np.array([
    [Ex*A, 0, 0, 0],
    [0, G*Ix, 0, 0],
    [0, 0, Ex*Iy, 0],
    [0, 0, 0, Ex*Iz]
])


def solve(n, angles, celltype=B2):
    # space
    GlobalFrame = StandardFrame(dim=3)
    TargetFrame = GlobalFrame.rotate('Body', angles, 'XYZ', inplace=False)

    # mesh
    p0 = np.array([0., 0., 0.])
    p1 = np.array([L, 0., 0.])
    coords = linspace(p0, p1, n+1)
    points = PointCloud(coords, frame=TargetFrame)
    coords = points.show(GlobalFrame)
    topo = np.zeros((n, 2), dtype=int)
    topo[:, 0] = np.arange(n)
    topo[:, 1] = np.arange(n) + 1
    if celltype.NNODE == 3:
        coords, topo = L2_to_L3(coords, topo)
    i_first = index_of_closest_point(coords, np.array([0., 0., 0.]))
    i_last = index_of_furthest_point(coords, np.array([0., 0., 0.]))

    # essential boundary conditions
    fixity = np.zeros((coords.shape[0], 6)).astype(bool)
    fixity[i_first, :] = True
    fixity = fixity.astype(float) * penalty

    # natural boundary conditions
    nodal_loads = np.zeros((coords.shape[0], 6))
    values = Vector([0, 0, Fz], frame=TargetFrame).show(GlobalFrame)
    nodal_loads[i_last, :3] = values

    # pointdata
    pd = PointData(coords=coords, frame=GlobalFrame,
                   loads=nodal_loads, fixity=fixity)

    # celldata
    frames = repeat(TargetFrame.axes, topo.shape[0])
    cd = celltype(topo=topo, frames=frames)

    # set up mesh and structure
    mesh = LineMesh(pd, cd, model=Hooke, frame=GlobalFrame)
    structure = Structure(mesh=mesh)

    structure.linsolve()

    dofsol = structure.nodal_dof_solution(store='dofsol')
    u = np.zeros(6)
    u[:3] = Vector(dofsol[i_last, :3], frame=GlobalFrame).show(TargetFrame)
    u[3:] = Vector(dofsol[i_last, 3:], frame=GlobalFrame).show(TargetFrame)

    reactions = structure.reaction_forces()
    r = np.zeros(6)
    r[:3] = Vector(reactions[i_first, :3], frame=GlobalFrame).show(TargetFrame)
    r[3:] = Vector(reactions[i_first, 3:], frame=GlobalFrame).show(TargetFrame)

    forces = structure.internal_forces()
    f = np.zeros(6)
    f[:3] = forces[0, 0, :3]
    f[3:] = forces[0, 0, 3:]

    return u, r, f


Analytical Solutions

In [4]:
UZ = Fz * L**3 / (3 * Ex * Iz)
UYY = -Fz * L**2 / (2 * Ex * Iz)
RZ = -Fz
RYY = Fz * L
FZ = Fz
FYY = -Fz * L


Testing Frame Objectivity

In [5]:
for i in range(3):
    angles = np.random.rand(3) * np.pi * 2
    u, r, f = solve(n, angles, B2)
    UZ_fem, UYY_fem = u[2], u[4]
    RZ_fem, RYY_fem = r[2], r[4]
    FZ_fem, FYY_fem = f[2], f[4]
    print("UZ  | Analytic : {}, FEM : {}".format(f2s(UZ), f2s(UZ_fem)))
    print("UYY | Analytic : {}, FEM : {}".format(f2s(UYY), f2s(UYY_fem)))
    print("RZ  | Analytic : {}, FEM : {}".format(f2s(RZ), f2s(RZ_fem)))
    print("RYY | Analytic : {}, FEM : {}".format(f2s(RYY), f2s(RYY_fem)))
    print("FZ  | Analytic : {}, FEM : {}".format(f2s(FZ), f2s(FZ_fem)))
    print("FYY | Analytic : {}, FEM : {}".format(f2s(FYY), f2s(FYY_fem)))
    print("\n")


UZ  | Analytic : 0.95245, FEM : 0.95245
UYY | Analytic : -0.00142867, FEM : -0.00142867
RZ  | Analytic : -1000, FEM : -1000
RYY | Analytic : 1e+06, FEM : 1e+06
FZ  | Analytic : 1000, FEM : 1000
FYY | Analytic : -1e+06, FEM : -1e+06


UZ  | Analytic : 0.95245, FEM : 0.95245
UYY | Analytic : -0.00142867, FEM : -0.00142867
RZ  | Analytic : -1000, FEM : -1000
RYY | Analytic : 1e+06, FEM : 1e+06
FZ  | Analytic : 1000, FEM : 1000
FYY | Analytic : -1e+06, FEM : -1e+06


UZ  | Analytic : 0.95245, FEM : 0.95245
UYY | Analytic : -0.00142867, FEM : -0.00142867
RZ  | Analytic : -1000, FEM : -1000
RYY | Analytic : 1e+06, FEM : 1e+06
FZ  | Analytic : 1000, FEM : 1000
FYY | Analytic : -1e+06, FEM : -1e+06




In [6]:
for i in range(3):
    angles = np.random.rand(3) * np.pi * 2
    u, r, f = solve(n, angles, B3)
    UZ_fem, UYY_fem = u[2], u[4]
    RZ_fem, RYY_fem = r[2], r[4]
    FZ_fem, FYY_fem = f[2], f[4]
    print("UZ  | Analytic : {}, FEM : {}".format(f2s(UZ), f2s(UZ_fem)))
    print("UYY | Analytic : {}, FEM : {}".format(f2s(UYY), f2s(UYY_fem)))
    print("RZ  | Analytic : {}, FEM : {}".format(f2s(RZ), f2s(RZ_fem)))
    print("RYY | Analytic : {}, FEM : {}".format(f2s(RYY), f2s(RYY_fem)))
    print("FZ  | Analytic : {}, FEM : {}".format(f2s(FZ), f2s(FZ_fem)))
    print("FYY | Analytic : {}, FEM : {}".format(f2s(FYY), f2s(FYY_fem)))
    print("\n")


UZ  | Analytic : 0.95245, FEM : 0.95245
UYY | Analytic : -0.00142867, FEM : -0.00142867
RZ  | Analytic : -1000, FEM : -1000
RYY | Analytic : 1e+06, FEM : 1e+06
FZ  | Analytic : 1000, FEM : 1000
FYY | Analytic : -1e+06, FEM : -1e+06


UZ  | Analytic : 0.95245, FEM : 0.95245
UYY | Analytic : -0.00142867, FEM : -0.00142867
RZ  | Analytic : -1000, FEM : -1000
RYY | Analytic : 1e+06, FEM : 1e+06
FZ  | Analytic : 1000, FEM : 1000
FYY | Analytic : -1e+06, FEM : -1e+06


UZ  | Analytic : 0.95245, FEM : 0.95245
UYY | Analytic : -0.00142867, FEM : -0.00142867
RZ  | Analytic : -1000, FEM : -1000
RYY | Analytic : 1e+06, FEM : 1e+06
FZ  | Analytic : 1000, FEM : 1000
FYY | Analytic : -1e+06, FEM : -1e+06


